# Single image super-resolution demonstration

Select GPU hardware acceleration (TPU not supported by TensorFlow 2) in Runtime > Change runtime type

In [0]:
%tensorflow_version 2.x

from google.colab import drive
drive.mount('/content/gdrive')

## Repository clone and dataset download (first time only)

In [0]:
%mkdir gdrive/My Drive/Part_II_Dissertation
%cd gdrive/My Drive/Part_II_Dissertation
!git clone https://github.com/EmptyJackson/video-super-resolution

In [0]:
%cd video-super-resolution

## Repository update (subsequent uses)

In [0]:
%cd gdrive/My Drive/Part_II_Dissertation/video-super-resolution

In [0]:
!git pull

In [0]:
!python src/tools/crop_image.py data/div2k/DIV2K_train_HR/0002.png lr_crop.png 240
!python src/upscale_image.py checkpoints/fsrcnn_x2_240p/ 55 lr_crop.png hr_crop.png

In [0]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(25,20))
fig.add_subplot(1, 2, 1)
im = plt.imread('lr_crop.png')
plt.axis('off')
plt.imshow(im)
fig.add_subplot(1, 2, 2)
im = plt.imread('hr_crop.png')
plt.axis('off')
plt.imshow(im)
plt.subplots_adjust(wspace=0.1)
plt.show()